### Project Euler Problem: 102

Three distinct points are plotted at random on a Cartesian plane,<br>for which -1000 ≤ x, y ≤ 1000, such that a triangle is formed.

Consider the following two triangles:

`A(-340,495), B(-153,-910), C(835,-947)`

`X(-175,41), Y(-421,-714), Z(574,-645)`

It can be verified that triangle `ABC` contains the origin, whereas triangle `XYZ` does not.

Using triangles.txt, a text file containing the co-ordinates of one thousand "random" triangles,<br>find the number of triangles for which the interior contains the origin.

### Solution: ###

The goal is to build a set $\Delta = \{t \mid t \in triangles \land \mathbb{0} \in t \}$ of those triangles which include the origin.<br>
Let's notice a few facts about the problem:
- Any triangle, which contains $\mathbb{0}$ in its interior, extends to all four quadrants of its graph.
- If $\mathbb{0}$ is an endpoint or a point on an edge, then the triangle is contained in the $\Delta$.
- Any edge which belongs to cater-corner quadrants either passes through a third quadrant or through $\mathbb{0}$.
- Given a line through cater-corner quadrants, the x and y -intercepts determine a third quadrant.
- If all of the endpoints of a triangle are in the same quadrant, then the triangle does not contain $\mathbb{0}$.
- Vertical edges will have infinite slope.

First, the data needs to be imported and stored as a usable data structure.<br>
I begin by importing `triangles.txt` as a csv and representing each triangle as a list of pairs.

In [53]:
import qualified Data.ByteString.Lazy as BL
import Data.Vector (Vector, empty, toList)
import Data.Either.Extra (fromRight)
import Data.List
import Data.Csv

csv <- BL.readFile "triangles.txt"
parsedCsv = decode NoHeader csv :: Either String (Vector [Integer])

triangles = map f $ toList.fromRight empty $ parsedCsv
    where
        f [a,b,c,d,e,f] = [(a,b),(c,d),(e,f)]

Next, I assign `[(Integer, Integer)]` a type synonym `Triangle`,<br>
create a datatype `Quadrant`, and give this datatype some accessor methods.<br>

In [54]:
type Triangle = [(Integer, Integer)]
data Quadrant = I | II | III | IV deriving (Eq, Show)

toQuad :: Triangle -> [Quadrant]
toQuad [] = []
toQuad (pt:pts) = f pt : toQuad pts
  where
    f (x,y) | and [x >= 0, y >= 0] = I
    f (x,y) | and [x <= 0, y >= 0] = II
    f (x,y) | and [x <= 0, y <= 0] = III
    f (x,y) | otherwise = IV

coords :: Triangle -> [Integer]
coords [(a,b),(c,d),(e,f)] = [a,b,c,d,e,f]

Next come a slew of functions characterizing the observations above.

In [55]:
onZero :: Triangle -> Bool -- endpoint is at (0,0)
onZero = any (== 0).coords

verticalEdge :: Triangle -> Bool -- same Xs
verticalEdge [] = False
verticalEdge ((p,q):ts) | any (== p) $ map fst ts = True
                        | otherwise = verticalEdge ts 

sameQuadrant :: Triangle -> Bool -- endpoints are all in same Quadrant.
sameQuadrant pts = f.toQuad $ pts
  where
    f (pt:pts) = all (== pt) pts

Now, I employ a sieving strategy, processing the list recursively in tiers. The sieving process relies on<br> `partition :: (a -> Bool) -> [a] -> ([a], [a])`, which lives in `Data.List`.<br>
The left list contains those elements for which the boolean-valued function is `True`, the other `False`.

In [60]:
partsSameQuad, partsOnVerts, partsOnZero :: [Triangle] -> ([Triangle], [Triangle])
partsSameQuad = partition sameQuadrant
partsOnVerts = partition verticalEdge
partsOnZero = partition onZero

1) separately check verticalEdges first as they will not have slopes.<br>
2) include any with a zero endpoint.<br>
3) remove if every edge is in the same quadrant, because it doesn't contain 0.<br>



In [61]:
quadI :: Triangle -> Bool -- endpoint ++ or II->IV line passes zero or higher.
quadI triangle = any (== I) $ toQuad triangle

-- hmmm. x- y- intercepts for lines, but these are segments.